In [ ]:
import sys
# This makes the gdc19 package avaialble
sys.path.append('../../')
import gdc19

In [ ]:
import numpy as np
import verde as vd
import pandas as pd
import vtki
import PVGeo
import discretize

**gCBGA**: Complete Bouguer anomaly

In [ ]:
name = 'gCBGA'
# load the gravity data
df = pd.read_csv(gdc19.get_gravity_path('Utah_FORGE_Gravity_Data.txt'))
ref = ['easting', 'northing', 'HAE']

In [ ]:
# Fit a trend

coordinates = np.array((df[ref[0]].values.astype(float), df[ref[1]].values.astype(float)))
values = df[name]


trend = vd.Trend(degree=2).fit(coordinates, values)
print(trend.coef_)

In [ ]:
trend_values = trend.predict(coordinates)
residuals = values - trend_values

In [ ]:
results = pd.DataFrame(data=df[ref])
results['residuals'] = residuals
results.to_csv(gdc19.get_gravity_path('gravity-residuals.csv'))
results.head()

In [ ]:
# save for SimPEG
results['std'] = np.full(len(results), 0.05)
with open('grav-inversion/grav-obs.grv', 'w') as f:
    f.write('{}\n'.format(len(results)))
    results.to_csv(f, sep='\t', header=False,index=False)

# Plot it

In [ ]:
topo = gdc19.load_topo()
topo

In [ ]:
topodf = pd.DataFrame(data=topo.points)
with open('grav-inversion/land-surface.topo', 'w') as f:
    f.write('{}\n'.format(len(topodf)))
    topodf.to_csv(f, sep=' ', header=False, index=False)

In [ ]:
poly = PVGeo.pointsToPolyData(results)
poly.copy().plot(scalars=values, stitle=name)

In [ ]:
poly.plot(scalars='residuals')

In [ ]:
p = vtki.Plotter(notebook=0)
p.add_mesh(poly)
p.add_mesh(topo)
p.show_grid()
p.show()

In [ ]:
p.add_mesh(vtki.Box(gdc19.get_roi_bounds()).outline(), 
           color='red', name='box', opacity=0.5,
           line_width=10)